In [1]:
import os
import pandas as pd

from utils import get_years_renewables, find_special_days, format_optim_repr_days, launch_optim_repr_days, parse_repr_days, format_epm_phours, format_epm_pvreprofile, format_data_energy

### What this script does ?

1. Extract renewables generation data from Ninja API.
2. Calculate representative year among historical data.
3. Calculate special and representatative days within this year.
4. Export pHours, pVREgen in EPM format.

### User input parameters (to manually change)

In [2]:
name_data = 'guinea' # Name of the zone, used to name the folder where the data is stored
extract_renewables = {'solar': False, 'wind': False} # Set to True to extract data from Renewables Ninja

### Create the folder structure

In [3]:
folder_raw_data = 'data'
# Make folder
if not os.path.exists(folder_raw_data):
    os.makedirs(folder_raw_data)
folder_raw_data = os.path.join(folder_raw_data, name_data)
if not os.path.exists(folder_raw_data):
    os.makedirs(folder_raw_data)
    
folder_process_data = 'parse_data'
# Make folder
if not os.path.exists(folder_process_data):
    os.makedirs(folder_process_data)
folder_process_data = os.path.join(folder_process_data, name_data)
if not os.path.exists(folder_process_data):
    os.makedirs(folder_process_data)

### 1. Extract data from Renewables Ninja

#### User input parameters (to manually change) 

Do not use this cell if data is already extracted

In [4]:
locations = [(10.1154, -13.1917)] # List of (latitude, longitude) tuples
power_types = ['solar', 'wind'] # 'wind' or 'solar'
start_year = 1980 # earliest year is 1980
end_year = 2024

In [5]:
for power_type in power_types:
    if extract_renewables[power_type]:
        get_years_renewables(locations=locations, power_type=power_type, start_year=start_year, 
                            end_year=end_year, name_data=name_data, output=folder_raw_data)

### 2. Process data

#### User input parameters (to manually change) 

In [6]:
filenames = {'PV': os.path.join(folder_raw_data, 'data_{}_solar.csv'.format(name_data)),
             'Wind': os.path.join(folder_raw_data, 'data_{}_wind.csv'.format(name_data))}

In [7]:
# Read, select representative year and format the data from .csv files
df_energy = format_data_energy(filenames)
# Find special days and remove them
special_days, df_energy = find_special_days(df_energy)
# Format the data (including correlation calculation) and save it in a .csv file
_, path_data_file = format_optim_repr_days(df_energy, name_data, folder_process_data)
# Launch the optimization to find the representative days
launch_optim_repr_days(path_data_file, folder_process_data)
# Get the results
repr_days = parse_repr_days(folder_process_data, special_days)
# Format the data to be used in EPM
format_epm_phours(repr_days, folder_process_data, name_data=name_data)
format_epm_pvreprofile(df_energy, repr_days, folder_process_data, name_data=name_data)


Annual capacity factor (%): PV      0.159687
Wind    0.140683
dtype: float64
File saved at: parse_data/guinea/data_formatted_optim_guinea.csv
Command to execute: ['gams', '/Users/lucas/Documents/World Bank/Projects/RepresentativeDays/gams/OptimizationModel.gms', '--data /Users/lucas/Documents/World Bank/Projects/RepresentativeDays/parse_data/guinea/data_formatted_optim_guinea.csv', '--settings /Users/lucas/Documents/World Bank/Projects/RepresentativeDays/gams/settings.csv', '--N 3']
Number of days: 59
Total weight: 390.0
File saved at: parse_data/guinea/pHours_guinea.csv
Number of hours: 33384
File saved at: parse_data/guinea/pVREProfile_guinea.csv


/opt/anaconda3/envs/gamsenv/lib/python3.8/site-packages/gams/transfer/containers/_container.py:73: UserWarning: The GAMS version (48.2.0) differs from the API version (47.6.0).
  ws = GamsWorkspace()
/opt/anaconda3/envs/gamsenv/lib/python3.8/site-packages/gams/core/numpy/gams2numpy.py:45: UserWarning: The GAMS version (48.2.0) differs from the API version (47.6.0).
  ws = GamsWorkspace(system_directory=system_directory)


In [ ]:
format_epm_phours(repr_days, folder_process_data, name_data=name_data)
format_epm_pvreprofile(df_energy, repr_days, folder_process_data, name_data=name_data)

File saved at: parse_data/guinea/pHours_guinea.csv
Number of hours: 33384
File saved at: parse_data/guinea/pVREProfile_guinea.csv
